In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/COL828_Assignment_1/Final Train Directory'
VAL_DIR = '/content/drive/MyDrive/COL828_Assignment_1/Final Validation directory'

train_datagen = ImageDataGenerator(
    rotation_range = 0.1,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=None
)

test_datagen = ImageDataGenerator(
    preprocessing_function=None
)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=(256,256),
    batch_size=64,
    class_mode='binary'
)


validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=(256,256),
    batch_size=16,
    class_mode='binary'
)

In [ ]:
def exponential_decay(lr, s):
    def exponential_decay_fn(epoch):
        return lr * 0.1
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.001, 5) # when i run it for 50 epochs

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
efficientmodel = EfficientNetV2S(weights = "imagenet", include_top = False,
                 input_tensor = Input(shape = (256,256,3)))
outputs = efficientmodel.output
outputs = Flatten(name = "flatten")(outputs)
outputs = Dropout(0.2)(outputs)
outputs = Dense(1, activation = "sigmoid")(outputs)

model = Model(inputs=efficientmodel.input, outputs=outputs)

for layers in efficientmodel.layers:
  layers.trainable = False

model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
model.summary()

history=model.fit(train_generator,
                        validation_data=validation_generator,
                        epochs = 30,
                        verbose = 1,
                         callbacks=lr_scheduler, steps_per_epoch = 20,
                         validation_steps = 5)
model.save('EfficientNet1.h5')

In [ ]:
# plot model performance
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(history.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()